### Jupyter Notebook for the Implemntation and training of the network based on the papet
"Research on the thermal-fluid coupling in the growth process of Czochralski silicon single crystals based on an improved physics-informed neural network" 

Next steps is in handling the CFD simulations data either from COMSOL or snythetic data generation 

# 1: Introduction & Theory
# Research on Thermal-Fluid Coupling in Czochralski Crystal Growth via SI-LB-PINN

**SI-LB-PINN** (Spatial-Information + Loss-Balancing Physics-Informed Neural Network) as described in DOI: [10.1063/5.0271778](https://doi.org/10.1063/5.0271778).

### The Czochralski (CZ) Process
The CZ process is the primary method for growing single-crystal silicon. It involves complex thermal-fluid coupling:
1. **Navier-Stokes Equations:** Govern the melt flow.
2. **Energy Equation:** Governs heat transfer.
3. **Boussinesq Approximation:** Usually used to couple temperature and flow via buoyancy.

### Key Innovations
1. **Spatial-Information (SI) Layer:** Standard MLPs often struggle with high-frequency spatial features. The SI layer injects raw coordinates $(x, y)$ into every hidden layer using learned gating mechanisms:
   $$H_{l} = \sigma( (1-Z) \odot M(X) + Z \odot N(X) + W H_{l-1} + b )$$
2. **Loss-Balancing (LB):** PINNs involve multiple loss terms (Continuity, Momentum, Energy, BCs). This implementation uses **Uncertainty Weighting** (adaptive weights) to automatically balance these terms during training.

In [ ]:
## Imports and Hyperparameters
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt
import time

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Physics & Network Params
config = {
    "nu": 1e-3,      # Kinematic viscosity
    "kappa": 1e-3,   # Thermal diffusivity
    "lr": 0.001,
    "epochs": 5000,
    "hidden_layers": 5,
    "hidden_units": 128
}

In [ ]:
## The Spatial-Information (SI) Architecture

class SILayer(nn.Module):
    """
    Injects spatial coordinates into the hidden state to preserve 
    spatial gradients better than standard MLPs.
    """
    def __init__(self, in_features, out_features, input_dim=2):
        super().__init__()
        self.lin = nn.Linear(in_features, out_features)
        self.M = nn.Sequential(nn.Linear(input_dim, out_features), nn.Tanh())
        self.N = nn.Sequential(nn.Linear(input_dim, out_features), nn.Tanh())
        self.Z = nn.Sequential(nn.Linear(in_features, out_features), nn.Sigmoid())

    def forward(self, H_prev, X):
        # Mx and Nx are spatial feature encoders
        Mx = self.M(X)
        Nx = self.N(X)
        # Zx is a gating mechanism based on the previous hidden state
        Zx = self.Z(H_prev)
        # Combine using the gate
        H = (1 - Zx) * Mx + Zx * Nx + self.lin(H_prev)
        return torch.tanh(H)

class SI_LB_PINN(nn.Module):
    def __init__(self, input_dim=2, hidden_units=128, hidden_layers=5, output_dim=4):
        super().__init__()
        self.input_lin = nn.Linear(input_dim, hidden_units)
        self.silayers = nn.ModuleList([
            SILayer(hidden_units, hidden_units, input_dim) 
            for _ in range(hidden_layers)
        ])
        self.out = nn.Linear(hidden_units, output_dim)

    def forward(self, x):
        H = torch.tanh(self.input_lin(x))
        for layer in self.silayers:
            H = layer(H, x)
        return self.out(H) # Returns [u, v, T, p]


In [ ]:
## Defining the Physics (PDE Residuals) the steady-state Navier-Stokes and Energy equations.

def pde_residuals(model, x, nu, kappa):
    x.requires_grad_(True)
    out = model(x)
    u, v, T, p = out[:, 0:1], out[:, 1:2], out[:, 2:3], out[:, 3:4]

    # Helper for gradients
    def grad(phi, x):
        return autograd.grad(phi, x, grad_outputs=torch.ones_like(phi), 
                             create_graph=True, retain_graph=True)[0]

    du = grad(u, x)
    dv = grad(v, x)
    dT = grad(T, x)
    dp = grad(p, x)

    u_x, u_y = du[:, 0:1], du[:, 1:2]
    v_x, v_y = dv[:, 0:1], dv[:, 1:2]
    T_x, T_y = dT[:, 0:1], dT[:, 1:2]
    p_x, p_y = dp[:, 0:1], dp[:, 1:2]

    # Second derivatives
    u_xx = grad(u_x, x)[:, 0:1]
    u_yy = grad(u_y, x)[:, 1:2]
    v_xx = grad(v_x, x)[:, 0:1]
    v_yy = grad(v_y, x)[:, 1:2]
    T_xx = grad(T_x, x)[:, 0:1]
    T_yy = grad(T_y, x)[:, 1:2]

    # Residuals
    r_cont = u_x + v_y
    r_momx = (u*u_x + v*u_y) + p_x - nu*(u_xx + u_yy)
    r_momy = (u*v_x + v*v_y) + p_y - nu*(v_xx + v_yy)
    r_energy = (u*T_x + v*T_y) - kappa*(T_xx + T_yy)

    return r_cont, r_momx, r_momy, r_energy

In [ ]:
## Adaptive Training Loop
model = SI_LB_PINN().to(device)
# 7 terms: 4 PDE + 3 Boundary Conditions
log_vars = nn.Parameter(torch.zeros(7, device=device), requires_grad=True)
optimizer = torch.optim.Adam(list(model.parameters()) + [log_vars], lr=config['lr'])

# Placeholder for boundary data sampling functions (defined in your code)
# ... insert sample_interior, sample_boundary, bc_values functions here ...

for epoch in range(config['epochs']):
    optimizer.zero_grad()
    
    # 1. PDE Loss
    X_int = sample_interior(2000)
    res = pde_residuals(model, X_int, config['nu'], config['kappa'])
    pde_losses = torch.stack([torch.mean(r**2) for r in res])
    
    # 2. BC Loss
    X_bc = sample_boundary(500)
    pred_bc = model(X_bc)
    true_bc = bc_values(X_bc)
    bc_losses = torch.stack([
        torch.mean((pred_bc[:, 0:1] - true_bc['u'])**2),
        torch.mean((pred_bc[:, 1:2] - true_bc['v'])**2),
        torch.mean((pred_bc[:, 2:3] - true_bc['T'])**2)
    ])
    
    # 3. Total Balanced Loss
    all_losses = torch.cat([pde_losses, bc_losses])
    loss = torch.sum(torch.exp(-log_vars) * all_losses + log_vars)
    
    loss.backward()
    optimizer.step()
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}: Loss {loss.item():.5f}")


# Part 2: Training on CFD Data (COMSOL)

The paper improves PINN performance by using data-informed training. Here is how to handle that.

### 1. Where to obtain Crystal Growth Data?
*   **Academic Databases:** Search for the **"Czochralski benchmark"** dataset. There are classic papers (e.g., Wheeler, 1990) that provide standard solutions for silicon melt.
*   **COMSOL Application Gallery:** Look for the **"Czochralski Crystal Growth"** model in the Heat Transfer or CFD module.
*   **OpenFOAM:** Use the `buoyantSimpleFoam` solver to simulate a rotating cylinder in a heated crucible.

### 2. How to export from COMSOL for PINNs
To use COMSOL data in PyTorch:
1.  Run your simulation in COMSOL.
2.  Go to **Results > Export > Data**.
3.  Select the entire domain.
4.  Export coordinates (`x, y`) and variables (`u, v, T, p`).
5.  Set the format to **CSV** or **Text**.
6.  **Crucial:** Ensure the data is nondimensionalized to match your PINN units (usually mapping the geometry to a $[0, 1]$ range).

### 3. Modifying the Code for Data Training
adding  `L_data` term to your loss function:


In [ ]:
# Assuming you loaded COMSOL data into tensors: X_data, U_data (u,v,T,p)
def train_with_data(model, X_data, U_data):
    # ... inside loop ...
    pred_data = model(X_data)
    L_data = torch.mean((pred_data - U_data)**2)
    
    # Add L_data to the balanced loss stack
    # loss = sum(exp(-log_v)*L + log_v) + L_data 


### 4. How to make Synthetic Data
If you don't have COMSOL, you can generate "pseudo-physical" synthetic data using:
1.  **Analytical Manufactured Solutions:** Define a function that looks like a melt (e.g., swirling vortices using stream functions) and calculate the source terms needed to make that function satisfy the PDE.
2.  **Simplified Solvers:** Use a coarse-grid Finite Difference method in Python (`numpy` or `scipy`) to solve a 2D lid-driven cavity problem with a temperature gradient.
3.  **Pre-trained PINN:** Train a standard PINN for 10,000 epochs, then use its output (filtered for high-confidence regions) as "noisy labels" for the SI-LB-PINN to show how the improved architecture converges faster/better.

### Recommendations for the Czochralski Case:
*   **Rotation:** In CZ growth, the crystal and crucible rotate. You must add the **Coriolis force** and **Centrifugal force** terms to your momentum residuals.
*   **Moving Boundary:** The crystal grows over time. For a simple PINN, start with a **Fixed Domain** (steady state) before attempting a time-dependent moving boundary.